In [1]:
from pathlib import Path
import pandas as pd
import re

# Try a few likely relative locations for the data directory (normally ../Data/demand_prices)
candidates = [Path('../Data/demand_prices')]
data_dir = None
for p in candidates:
    if p.exists() and p.is_dir():
        data_dir = p
        break

if data_dir is None:
    raise FileNotFoundError('Could not find the Data/demand_prices directory.')

# Columns to exclude entirely (never load them)
columns_to_drop = [
    "scottish_transfer",
    "eleclink_flow",
    "greenlink_flow",
    "viking_flow",
    "nsl_flow",
    "source_file"
]

# Read and filter CSV files based on filename year
all_files = sorted(data_dir.glob('*.csv'))
files = []

for f in all_files:
    match = re.search(r'(\d{4})', f.name)
    if match:
        year = int(match.group(1))
        if year >= 2009:
            files.append(f)
        else:
            print(f"Skipping {f.name}: year {year} < 2009")
    else:
        print(f"Skipping {f.name}: no 4-digit year found")

if not files:
    raise FileNotFoundError("No CSV files from 2009 onward found.")

dfs = []
headers_list = []

for f in files:
    try:
        # Read header exactly as-is (original column names)
        raw_cols = pd.read_csv(f, nrows=0).columns

        # Lowercase version for checking exclusions
        raw_cols_lower = raw_cols.str.lower()

        # Build list of columns we want to load (original names!)
        usecols = [orig for orig, low in zip(raw_cols, raw_cols_lower)
                if low not in columns_to_drop]

        # Now read CSV with selected columns
        df = pd.read_csv(f, usecols=usecols)

        # After loading → convert to lowercase
        df.columns = df.columns.str.lower()

    except Exception as e:
        print(f'Failed to read {f}: {e}')
        continue

    if len(df) > 0:
        header_row = df.iloc[[0]].copy()
        header_row['source_file'] = f.name
        headers_list.append(header_row)
    else:
        headers_list.append(pd.DataFrame({'source_file': [f.name]}))

    dfs.append(df)

# Concatenate
if dfs:
    demand_df = pd.concat(dfs, ignore_index=True)
else:
    demand_df = pd.DataFrame()

# Build headers_demand
if headers_list:
    headers_demand = pd.concat(headers_list, ignore_index=True, sort=False)
    headers_demand.columns = headers_demand.columns.str.lower()
else:
    headers_demand = pd.DataFrame()

print(f'Combined {len(dfs)} files from {data_dir} — total rows: {len(demand_df)}')
print(f'Collected first-row headers from {len(headers_list)} files — headers_demand shape: {headers_demand.shape}')

print('\nSample of combined demand_df:')
display(demand_df.head())
demand_df.info()

print('\nSample of headers_demand (first row from each CSV):')
display(headers_demand.head(25))
headers_demand.info()


Skipping cleaned_data_neso_demand_2001_2025.csv: year 2001 < 2009
Skipping demanddata_2001.csv: year 2001 < 2009
Skipping demanddata_2002.csv: year 2002 < 2009
Skipping demanddata_2003.csv: year 2003 < 2009
Skipping demanddata_2004.csv: year 2004 < 2009
Skipping demanddata_2005.csv: year 2005 < 2009
Skipping demanddata_2006.csv: year 2006 < 2009
Skipping demanddata_2007.csv: year 2007 < 2009
Skipping demanddata_2008.csv: year 2008 < 2009
Failed to read ..\Data\demand_prices\demand_data_2009_2025.csv: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte
Combined 17 files from ..\Data\demand_prices — total rows: 293758
Collected first-row headers from 17 files — headers_demand shape: (17, 18)

Sample of combined demand_df:


,settlement_date,settlement_period,nd,tsd,england_wales_demand,embedded_wind_generation,embedded_wind_capacity,embedded_solar_generation,embedded_solar_capacity,non_bm_stor,pump_storage_pumping,ifa_flow,ifa2_flow,britned_flow,moyle_flow,east_west_flow,nemo_flow
0,01-JAN-2009,1,37910,38704,33939,54,1403,0,0,0,33,2002,0,0,-161,0,0
1,01-JAN-2009,2,38047,38964,34072,53,1403,0,0,0,157,2002,0,0,-160,0,0
2,01-JAN-2009,3,37380,38651,33615,53,1403,0,0,0,511,2002,0,0,-160,0,0
3,01-JAN-2009,4,36426,37775,32526,50,1403,0,0,0,589,1772,0,0,-160,0,0
4,01-JAN-2009,5,35687,37298,31877,50,1403,0,0,0,851,1753,0,0,-160,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293758 entries, 0 to 293757
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   settlement_date            293758 non-null  object
 1   settlement_period          293758 non-null  int64 
 2   nd                         293758 non-null  int64 
 3   tsd                        293758 non-null  int64 
 4   england_wales_demand       293758 non-null  int64 
 5   embedded_wind_generation   293758 non-null  int64 
 6   embedded_wind_capacity     293758 non-null  int64 
 7   embedded_solar_generation  293758 non-null  int64 
 8   embedded_solar_capacity    293758 non-null  int64 
 9   non_bm_stor                293758 non-null  int64 
 10  pump_storage_pumping       293758 non-null  int64 
 11  ifa_flow                   293758 non-null  int64 
 12  ifa2_flow                  293758 non-null  int64 
 13  britned_flow               293758 non-null  

,settlement_date,settlement_period,nd,tsd,england_wales_demand,embedded_wind_generation,embedded_wind_capacity,embedded_solar_generation,embedded_solar_capacity,non_bm_stor,pump_storage_pumping,ifa_flow,ifa2_flow,britned_flow,moyle_flow,east_west_flow,nemo_flow,source_file
0,01-JAN-2009,1,37910,38704,33939,54,1403,0,0,0,33,2002,0,0,-161,0,0,demanddata_2009.csv
1,01-JAN-2010,1,36453,37593,32391,906,1786,0,0,0,306,1977,0,0,-234,0,0,demanddata_2010.csv
2,01-JAN-2011,1,34606,35648,31058,484,1730,0,79,0,60,1939,0,0,-382,0,0,demanddata_2011.csv
3,01-JAN-2012,1,30263,31262,27228,1152,1866,0,1122,0,399,1746,0,993,0,0,0,demanddata_2012.csv
4,01-JAN-2013,1,30565,32789,27585,963,2085,0,2036,0,1123,1490,0,993,-251,-250,0,demanddata_2013.csv
5,01-JAN-2014,1,30008,31920,27048,1084,2434,0,3349,0,870,1990,0,1000,-157,-285,0,demanddata_2014.csv
6,01-JAN-2015,1,28726,29490,26351,2211,4039,0,5994,0,18,980,0,854,-25,-121,0,demanddata_2015.csv
7,01-JAN-2016,1,27700,28414,25315,1580,4013,0,9063,0,114,1756,0,870,31,105,0,demanddata_2016.csv
8,01-JAN-2017,1,27316,28013,24789,1634,4871,0,11503,0,97,923,0,857,295,233,0,demanddata_2017.csv
9,01-JAN-2018,1,25593,26501,23307,3064,5754,0,12916,0,308,1415,0,715,205,432,0,demanddata_2018.csv


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   settlement_date            17 non-null     object
 1   settlement_period          17 non-null     int64 
 2   nd                         17 non-null     int64 
 3   tsd                        17 non-null     int64 
 4   england_wales_demand       17 non-null     int64 
 5   embedded_wind_generation   17 non-null     int64 
 6   embedded_wind_capacity     17 non-null     int64 
 7   embedded_solar_generation  17 non-null     int64 
 8   embedded_solar_capacity    17 non-null     int64 
 9   non_bm_stor                17 non-null     int64 
 10  pump_storage_pumping       17 non-null     int64 
 11  ifa_flow                   17 non-null     int64 
 12  ifa2_flow                  17 non-null     int64 
 13  britned_flow               17 non-null     int64 
 14  moyle_flow  

In [2]:
def clean_settlement_date(df, col='settlement_date'):
    # df[col] = df[col].replace('########', np.nan) no longer necessary, correct in csv files
    d1 = pd.to_datetime(df[col], format='%Y-%m-%d', errors='coerce')
    d2 = pd.to_datetime(df[col], format='%d-%b-%y', errors='coerce')
    d3 = pd.to_datetime(df[col], format='%d-%b-%Y', errors='coerce')
    df[col] = d1.fillna(d2).fillna(d3)
    # df = df.dropna(subset=[col])
    df[col] = df[col].dt.strftime('%Y-%m-%d')
    return df

headers_demand = clean_settlement_date(headers_demand, col='settlement_date')

display(headers_demand.head(25))
headers_demand.info()

,settlement_date,settlement_period,nd,tsd,england_wales_demand,embedded_wind_generation,embedded_wind_capacity,embedded_solar_generation,embedded_solar_capacity,non_bm_stor,pump_storage_pumping,ifa_flow,ifa2_flow,britned_flow,moyle_flow,east_west_flow,nemo_flow,source_file
0,2009-01-01,1,37910,38704,33939,54,1403,0,0,0,33,2002,0,0,-161,0,0,demanddata_2009.csv
1,2010-01-01,1,36453,37593,32391,906,1786,0,0,0,306,1977,0,0,-234,0,0,demanddata_2010.csv
2,2011-01-01,1,34606,35648,31058,484,1730,0,79,0,60,1939,0,0,-382,0,0,demanddata_2011.csv
3,2012-01-01,1,30263,31262,27228,1152,1866,0,1122,0,399,1746,0,993,0,0,0,demanddata_2012.csv
4,2013-01-01,1,30565,32789,27585,963,2085,0,2036,0,1123,1490,0,993,-251,-250,0,demanddata_2013.csv
5,2014-01-01,1,30008,31920,27048,1084,2434,0,3349,0,870,1990,0,1000,-157,-285,0,demanddata_2014.csv
6,2015-01-01,1,28726,29490,26351,2211,4039,0,5994,0,18,980,0,854,-25,-121,0,demanddata_2015.csv
7,2016-01-01,1,27700,28414,25315,1580,4013,0,9063,0,114,1756,0,870,31,105,0,demanddata_2016.csv
8,2017-01-01,1,27316,28013,24789,1634,4871,0,11503,0,97,923,0,857,295,233,0,demanddata_2017.csv
9,2018-01-01,1,25593,26501,23307,3064,5754,0,12916,0,308,1415,0,715,205,432,0,demanddata_2018.csv


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   settlement_date            17 non-null     object
 1   settlement_period          17 non-null     int64 
 2   nd                         17 non-null     int64 
 3   tsd                        17 non-null     int64 
 4   england_wales_demand       17 non-null     int64 
 5   embedded_wind_generation   17 non-null     int64 
 6   embedded_wind_capacity     17 non-null     int64 
 7   embedded_solar_generation  17 non-null     int64 
 8   embedded_solar_capacity    17 non-null     int64 
 9   non_bm_stor                17 non-null     int64 
 10  pump_storage_pumping       17 non-null     int64 
 11  ifa_flow                   17 non-null     int64 
 12  ifa2_flow                  17 non-null     int64 
 13  britned_flow               17 non-null     int64 
 14  moyle_flow  

In [3]:
demand_df = clean_settlement_date(demand_df, col='settlement_date')

display(demand_df.head())
demand_df.info()

,settlement_date,settlement_period,nd,tsd,england_wales_demand,embedded_wind_generation,embedded_wind_capacity,embedded_solar_generation,embedded_solar_capacity,non_bm_stor,pump_storage_pumping,ifa_flow,ifa2_flow,britned_flow,moyle_flow,east_west_flow,nemo_flow
0,2009-01-01,1,37910,38704,33939,54,1403,0,0,0,33,2002,0,0,-161,0,0
1,2009-01-01,2,38047,38964,34072,53,1403,0,0,0,157,2002,0,0,-160,0,0
2,2009-01-01,3,37380,38651,33615,53,1403,0,0,0,511,2002,0,0,-160,0,0
3,2009-01-01,4,36426,37775,32526,50,1403,0,0,0,589,1772,0,0,-160,0,0
4,2009-01-01,5,35687,37298,31877,50,1403,0,0,0,851,1753,0,0,-160,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293758 entries, 0 to 293757
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   settlement_date            293758 non-null  object
 1   settlement_period          293758 non-null  int64 
 2   nd                         293758 non-null  int64 
 3   tsd                        293758 non-null  int64 
 4   england_wales_demand       293758 non-null  int64 
 5   embedded_wind_generation   293758 non-null  int64 
 6   embedded_wind_capacity     293758 non-null  int64 
 7   embedded_solar_generation  293758 non-null  int64 
 8   embedded_solar_capacity    293758 non-null  int64 
 9   non_bm_stor                293758 non-null  int64 
 10  pump_storage_pumping       293758 non-null  int64 
 11  ifa_flow                   293758 non-null  int64 
 12  ifa2_flow                  293758 non-null  int64 
 13  britned_flow               293758 non-null  

In [4]:
demand_df.to_csv("../Data/demand_prices/demand_data_2009_2025.csv", index=False, compression='gzip')